## LLM

In [1]:
import json

# Load configuration from JSON file
with open('../config.json') as config_file:
    config = json.load(config_file)

In [2]:
db_scheme = config.get("DB_SCHEME")
palm_api_key = config.get("API_KEY")
user = config.get("user")
password = config.get("password")
dbname = config.get("dbname")

In [3]:
from llama_index.llms.palm import PaLM
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    StorageContext,
    load_index_from_storage,
)

# use Huggingface embeddings
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

c:\Users\TurnerZ\Documents\GitHub\DBSage_Navigator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 743/743 [00:00<?, ?B/s] 
c:\Users\TurnerZ\Documents\GitHub\DBSage_Navigator\.venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\TurnerZ\AppData\Local\llama_index. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develop

In [4]:
model = PaLM(api_key=palm_api_key)
service_context = ServiceContext.from_defaults(
    llm=model,
    embed_model=embed_model,)

## Loading Data

In [5]:
from llama_index import download_loader

DatabaseReader = download_loader('DatabaseReader')

In [6]:
reader = DatabaseReader(
    scheme = "postgresql", # Database Scheme
    host = "localhost", # Database Host
    port = "5432", # Database Port
    user = user, # Database User
    password = password, # Database Password
    dbname = dbname, # Database Name
)

query = """
SELECT * FROM public.employee_data
"""

documents = reader.load_data(query=query)

## Indexing

In [7]:
import os

# check if storage already exists
if not os.path.exists("../dbsage_storage"):
    print('creating storage')  
    index = VectorStoreIndex.from_documents(
            documents, service_context=service_context
        )
    # store it for later
    index.storage_context.persist('../dbsage_storage')
    print('Done...')
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir="../dbsage_storage")
    index = load_index_from_storage(storage_context, service_context=service_context)

creating storage
Done...
